# Media Description Generator

ResNet18 architecture is used with ImageNet pretrained model, we also tried with Pretrained='False',but given the time constraints,we could not get a good convergence (see README), and pretrained model gave better result and converged early. 

Summary:
- Converting audio to 2D image spectrogram, so that we use  CNN classifier using vision based deep learing techniques.
- Use transfer learning by using ImageNet weight initialization.
- We are using FastAI library to build this multi class clasifier and then we call our next pretrained model **DeepSpeech** for caption generation.
- Implementation of LwLARP taken from [Dan Ellis](https://colab.research.google.com/drive/1AgPdhSp7ttY18O3fEoHOQKlt_3HJDLi8).




In [0]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import IPython
import IPython.display
import PIL
import os

In [0]:
from fastai.vision import *

In [0]:
#downloading dataset
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-IN,en;q=0.9,en-US;q=0.8,hi;q=0.7" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/10700/503808/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1580296888&Signature=JYVxUjlBN08F4EPTgrmhrTY2swSqc60S4V2IDfD5pQMf8rez1eOb9lGJn2o04JEIF1q9hWJXCeCD2zm9kC6tn4vivrEizrJ0Gs8ZFx%2BNDMagbOBJ6B6tatYHOodncAmHldL3eVcc86Q7VdRn55goP0%2FUBadwLQj1CKCgPI58ZK1hBKGtloOnSRo2ZTbrmVvKp2v7hVnrkAO8BmGJoWHMAbJU0JezEj2sJTN%2Bfxcs9QCAPJINJ5aDtlfhoNS8nLkmD0H5L1BhnAtT87n51%2BUpE81A2du07O7EEEw6K8OhrtZT5kqAPopgA20Mw5%2B7nSUJZFqaPIA%2FTeFEaQToIoKSZA%3D%3D&response-content-disposition=attachment%3B+filename%3Dfreesound-audio-tagging-2019.zip" -O "freesound-audio-tagging-2019.zip" -c

--2020-01-27 09:34:29--  https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/10700/503808/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1580296888&Signature=JYVxUjlBN08F4EPTgrmhrTY2swSqc60S4V2IDfD5pQMf8rez1eOb9lGJn2o04JEIF1q9hWJXCeCD2zm9kC6tn4vivrEizrJ0Gs8ZFx%2BNDMagbOBJ6B6tatYHOodncAmHldL3eVcc86Q7VdRn55goP0%2FUBadwLQj1CKCgPI58ZK1hBKGtloOnSRo2ZTbrmVvKp2v7hVnrkAO8BmGJoWHMAbJU0JezEj2sJTN%2Bfxcs9QCAPJINJ5aDtlfhoNS8nLkmD0H5L1BhnAtT87n51%2BUpE81A2du07O7EEEw6K8OhrtZT5kqAPopgA20Mw5%2B7nSUJZFqaPIA%2FTeFEaQToIoKSZA%3D%3D&response-content-disposition=attachment%3B+filename%3Dfreesound-audio-tagging-2019.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.127.128, 2a00:1450:4013:c00::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.127.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26154728032 (24G) [application/zip]
Saving to: ‘freesound-audio-tagging-2019.zip’

fr

In [0]:
!unzip "freesound-audio-tagging-2019.zip" 

Archive:  freesound-audio-tagging-2019.zip
  inflating: sample_submission.csv   
  inflating: test.zip                
  inflating: train_curated.csv       
  inflating: train_curated.zip       
  inflating: train_noisy.csv         
  inflating: train_noisy.zip         


In [0]:
!rm "freesound-audio-tagging-2019.zip" 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## File/folder definitions


In [0]:
DATA = Path('/content')
CSV_TRN_CURATED = DATA/'train_curated.csv'
CSV_SKELETON = DATA/'skeleton.csv'
TRN_CURATED = DATA/'train_curated'
TEST = DATA/'test'
WORK = Path('work')
IMG_TRN_CURATED = WORK/'image/trn_curated'
IMG_TEST = WORK/'image/test'
for folder in [WORK, IMG_TRN_CURATED, IMG_TEST]: 
    Path(folder).mkdir(exist_ok=True, parents=True)

df = pd.read_csv(CSV_TRN_CURATED)
test_df = pd.read_csv(CSV_SKELETON)

## Audio conversion to 2D spectrogram

Spectrogram creation inspired from https://github.com/daisukelab/ml-sound-classifier
- Size of each file will be 128 x L, L is audio seconds x 128; `[128, 256]  provided sound is 2s long.
- We now convert into spectrogram

In [0]:
import librosa
import librosa.display

def read_audio(conf, pathname, trim_long_data):
    y, sr = librosa.load(pathname, sr=conf.sampling_rate)
    # trim silence
    if 0 < len(y): # workaround: 0 length causes error
        y, _ = librosa.effects.trim(y) # trim, top_db=default(60)
    # make it unified length to conf.samples
    if len(y) > conf.samples: # long enough
        if trim_long_data:
            y = y[0:0+conf.samples]
    else: # pad blank
        padding = conf.samples - len(y)    # add padding at both ends
        offset = padding // 2
        y = np.pad(y, (offset, conf.samples - len(y) - offset), 'constant')
    return y

def audio_to_melspectrogram(conf, audio):
    spectrogram = librosa.feature.melspectrogram(audio, 
                                                 sr=conf.sampling_rate,
                                                 n_mels=conf.n_mels,
                                                 hop_length=conf.hop_length,
                                                 n_fft=conf.n_fft,
                                                 fmin=conf.fmin,
                                                 fmax=conf.fmax)
    spectrogram = librosa.power_to_db(spectrogram)
    spectrogram = spectrogram.astype(np.float32)
    return spectrogram

def show_melspectrogram(conf, mels, title='Log-frequency power spectrogram'):
    librosa.display.specshow(mels, x_axis='time', y_axis='mel', 
                             sr=conf.sampling_rate, hop_length=conf.hop_length,
                            fmin=conf.fmin, fmax=conf.fmax)
    plt.colorbar(format='%+2.0f dB')
    plt.title(title)
    plt.show()

def read_as_melspectrogram(conf, pathname, trim_long_data, debug_display=False):
    x = read_audio(conf, pathname, trim_long_data)
    mels = audio_to_melspectrogram(conf, x)
    if debug_display:
        IPython.display.display(IPython.display.Audio(x, rate=conf.sampling_rate))
        show_melspectrogram(conf, mels)
    return mels


class conf:
    # Preprocessing settings
    sampling_rate = 44100
    duration = 2
    hop_length = 347*duration # to make time steps 128
    fmin = 20
    fmax = sampling_rate // 2
    n_mels = 128
    n_fft = n_mels * 20
    samples = sampling_rate * duration

# example
#x = read_as_melspectrogram(conf, TRN_CURATED/'0006ae4e.wav', trim_long_data=False, debug_display=True)

## Converting 2D spectrogram data to three channel image files

 So that we can use normal CNN for classification



We are keeping data on memory instead of saving on disk for faster training.

In [0]:
def mono_to_color(X, mean=None, std=None, norm_max=None, norm_min=None, eps=1e-6):
    # Stack X as [X,X,X]
    X = np.stack([X, X, X], axis=-1)

    # Standardize
    mean = mean or X.mean()
    std = std or X.std()
    Xstd = (X - mean) / (std + eps)
    _min, _max = Xstd.min(), Xstd.max()
    norm_max = norm_max or _max
    norm_min = norm_min or _min
    if (_max - _min) > eps:
        # Scale to [0, 255]
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V

def convert_wav_to_image(df, source, img_dest):
    X = []
    for i, row in tqdm_notebook(df.iterrows()):
        x = read_as_melspectrogram(conf, source/str(row.fname), trim_long_data=False)
        x_color = mono_to_color(x)
        X.append(x_color)
    return X

X_train = convert_wav_to_image(df, source=TRN_CURATED, img_dest=IMG_TRN_CURATED)
X_test = convert_wav_to_image(test_df, source=TEST, img_dest=IMG_TEST)

## Specifying `open_image` for fast.ai library to load data from memory
 Also using random cropping of 1 sec, for data augmentation similar to images.

In [0]:
from fastai import *
from fastai.vision import *
from fastai.vision.data import *
import random

CUR_X_FILES, CUR_X = list(df.fname.values), X_train

def open_fat2019_image(fn, convert_mode, after_open)->Image:
    # open
    idx = CUR_X_FILES.index(fn.split('/')[-1])
    x = PIL.Image.fromarray(CUR_X[idx])
    # crop
    time_dim, base_dim = x.size
    crop_x = random.randint(0, time_dim - base_dim)
    x = x.crop([crop_x, 0, crop_x+base_dim, base_dim])    
    # standardize
    return Image(pil2tensor(x, np.float32).div_(255))

vision.data.open_image = open_fat2019_image

## Code for multi-label classification 

*Implementation of LwLARP taken from [Dan Ellis](https://colab.research.google.com/drive/1AgPdhSp7ttY18O3fEoHOQKlt_3HJDLi8).*

-

In [0]:
# from official code https://colab.research.google.com/drive/1AgPdhSp7ttY18O3fEoHOQKlt_3HJDLi8#scrollTo=cRCaCIb9oguU
def _one_sample_positive_class_precisions(scores, truth):
    """Calculate precisions for each true class for a single sample.

    Args:
      scores: np.array of (num_classes,) giving the individual classifier scores.
      truth: np.array of (num_classes,) bools indicating which classes are true.

    Returns:
      pos_class_indices: np.array of indices of the true classes for this sample.
      pos_class_precisions: np.array of precisions corresponding to each of those
        classes.
    """
    num_classes = scores.shape[0]
    pos_class_indices = np.flatnonzero(truth > 0)
    # Only calculate precisions if there are some true classes.
    if not len(pos_class_indices):
        return pos_class_indices, np.zeros(0)
    # Retrieval list of classes for this sample.
    retrieved_classes = np.argsort(scores)[::-1]
    # class_rankings[top_scoring_class_index] == 0 etc.
    class_rankings = np.zeros(num_classes, dtype=np.int)
    class_rankings[retrieved_classes] = range(num_classes)
    # Which of these is a true label?
    retrieved_class_true = np.zeros(num_classes, dtype=np.bool)
    retrieved_class_true[class_rankings[pos_class_indices]] = True
    # Num hits for every truncated retrieval list.
    retrieved_cumulative_hits = np.cumsum(retrieved_class_true)
    # Precision of retrieval list truncated at each hit, in order of pos_labels.
    precision_at_hits = (
            retrieved_cumulative_hits[class_rankings[pos_class_indices]] /
            (1 + class_rankings[pos_class_indices].astype(np.float)))
    return pos_class_indices, precision_at_hits


def calculate_per_class_lwlrap(truth, scores):
    """Calculate label-weighted label-ranking average precision.

    Arguments:
      truth: np.array of (num_samples, num_classes) giving boolean ground-truth
        of presence of that class in that sample.
      scores: np.array of (num_samples, num_classes) giving the classifier-under-
        test's real-valued score for each class for each sample.

    Returns:
      per_class_lwlrap: np.array of (num_classes,) giving the lwlrap for each
        class.
      weight_per_class: np.array of (num_classes,) giving the prior of each
        class within the truth labels.  Then the overall unbalanced lwlrap is
        simply np.sum(per_class_lwlrap * weight_per_class)
    """
    assert truth.shape == scores.shape
    num_samples, num_classes = scores.shape
    # Space to store a distinct precision value for each class on each sample.
    # Only the classes that are true for each sample will be filled in.
    precisions_for_samples_by_classes = np.zeros((num_samples, num_classes))
    for sample_num in range(num_samples):
        pos_class_indices, precision_at_hits = (
            _one_sample_positive_class_precisions(scores[sample_num, :],
                                                  truth[sample_num, :]))
        precisions_for_samples_by_classes[sample_num, pos_class_indices] = (
            precision_at_hits)
    labels_per_class = np.sum(truth > 0, axis=0)
    weight_per_class = labels_per_class / float(np.sum(labels_per_class))
    # Form average of each column, i.e. all the precisions assigned to labels in
    # a particular class.
    per_class_lwlrap = (np.sum(precisions_for_samples_by_classes, axis=0) /
                        np.maximum(1, labels_per_class))
    # overall_lwlrap = simple average of all the actual per-class, per-sample precisions
    #                = np.sum(precisions_for_samples_by_classes) / np.sum(precisions_for_samples_by_classes > 0)
    #           also = weighted mean of per-class lwlraps, weighted by class label prior across samples
    #                = np.sum(per_class_lwlrap * weight_per_class)
    return per_class_lwlrap, weight_per_class


# Wrapper for fast.ai library
def lwlrap(scores, truth, **kwargs):
    score, weight = calculate_per_class_lwlrap(to_np(truth), to_np(scores))
    return torch.Tensor([(score * weight).sum()])


###Transforms and Creating Learner for training

In [0]:
tfms = get_transforms(do_flip=True, max_rotate=0, max_lighting=0.1, max_zoom=0, max_warp=0.)
src = (ImageList.from_csv(WORK/'image', Path('../../')/CSV_TRN_CURATED, folder='trn_curated')
       .split_by_rand_pct(0.2)
       .label_from_df(label_delim=',')
)
data = (src.transform(tfms, size=128)
        .databunch(bs=64).normalize(imagenet_stats)
)

In [0]:
data.show_batch(3)

In [0]:
learn = cnn_learner(data, models.resnet18, pretrained=True, metrics=[lwlrap])
learn.unfreeze()

learn.lr_find(); learn.recorder.plot()

In [0]:
learn.fit_one_cycle(3, 1e-1)
learn.fit_one_cycle(7, 1e-2)

In [0]:
learn.lr_find(); learn.recorder.plot()

In [0]:
learn.fit_one_cycle(15,2e-3)

In [0]:
learn.lr_find(); learn.recorder.plot()

In [0]:
learn.fit_one_cycle(25, slice(1e-3, 2e-3))

In [0]:
learn.fit_one_cycle(5, slice(1e-4, 2e-3))

# Saving the model for later inference

In [0]:
learn.save('stage-2')
learn.export()